In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime

# Pergunta ao usuário qual produto deseja procurar
produto = input('Qual produto você deseja procurar na Boticário: ')

# Configuração do WebDriver
navegador = webdriver.Chrome()

# Maximiza a janela
navegador.maximize_window()

# URL da busca com o produto
url = f'https://www.boticario.com.br/busca?q={produto}'

# Abre a URL no navegador
navegador.get(url)

# Aceita cookies se necessário
try:
    wait = WebDriverWait(navegador, 10)
    botao_aceitar_cookies = wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler')))
    botao_aceitar_cookies.click()
except Exception as e:
    print('Não foi possível aceitar os cookies.')

# Encontra todos os elementos de classe 'showcase-image'
elementos_produtos = navegador.find_elements(By.CLASS_NAME, 'showcase-image')

# Lista para armazenar os resultados
resultados = []

# Itera sobre os elementos para obter informações
for i in range(len(elementos_produtos)):
    try:
        elemento = elementos_produtos[i]
        url_imagem = elemento.get_attribute('src')

        # Clicar na imagem para abrir detalhes do produto
        elemento.click()

        # Aguarda a visibilidade do título (h1)
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'h1')))
        elemento_h1 = navegador.find_element(By.TAG_NAME, 'h1')
        titulo_h1 = elemento_h1.text

        # Aguarda a visibilidade do preço
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'nproduct-price-value')))
        elemento_preco = navegador.find_element(By.CLASS_NAME, 'nproduct-price-value')
        preco_texto = elemento_preco.text.replace('R$', '').replace(',', '.').strip()

        # Converte para float e depois para inteiro
        preco = int(float(preco_texto))

        # Adiciona os resultados à lista
        resultados.append({
            'titulo': titulo_h1,
            'preco': preco,
            'avaliacao': i,  # Adicione a avaliação real, se disponível
            'link_imagem': url_imagem
        })

        # Voltar para a página de resultados
        navegador.back()

        # Aguarde a visibilidade dos elementos novamente
        wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'showcase-image')))
        elementos_produtos = navegador.find_elements(By.CLASS_NAME, 'showcase-image')

    except IndexError:
        print(f"Erro: Não foi possível acessar o índice {i} na lista. Provavelmente, não há mais elementos.")
        break

indices = []
titulos = []
precos = []
avaliacoes = []
links_imagem = []

# Loop para iterar sobre os resultados
for idx, resultado in enumerate(resultados, start=1):
    # Seu código para extrair os dados...

    # Exemplo: Se 'resultado' for um dicionário, você pode acessar as chaves assim:
    titulo = resultado.get('titulo', 'N/A')  # 'N/A' é um valor padrão caso 'titulo' não esteja presente
    preco = resultado.get('preco', 'N/A') 
    avaliacao = resultado.get('avaliacao', 'N/A')  
    url_imagem = resultado.get('link_imagem', 'N/A')

    # Adicionar dados às listas
    indices.append(idx)
    titulos.append(titulo)
    precos.append(preco)
    avaliacoes.append(avaliacao)
    links_imagem.append(url_imagem)

    # Exibir os dados
    print(f'{idx}: {titulo}, {preco}, {avaliacao}, {url_imagem}')

# Criar DataFrame a partir das listas
tabela = pd.DataFrame({
    'Índice': indices,
    'Título': titulos,
    'Preço': precos,
    'Avaliação': avaliacoes,
    'Link da Imagem': links_imagem
})

# Salvar o DataFrame em um arquivo Excel
data_atual = datetime.now().strftime("%Y-%m-%d")
nome_arquivo = f'{produto}_resultados_{data_atual}.xlsx'
tabela.to_excel(nome_arquivo, index=False)

# Exibir o DataFrame
print(tabela)

navegador.quit()


1: Pente Garfo Wakanda Forever, 39, 0, https://res.cloudinary.com/beleza-na-web/image/upload/w_1500,f_auto,fl_progressive,q_auto:eco,w_210,h_210/v1/imagens/products/B49917/Sophie_Pente_B49917.jpg
2: Escova Desembaraço Sophie, 39, 1, https://res.cloudinary.com/beleza-na-web/image/upload/w_1500,f_auto,fl_progressive,q_auto:eco,w_210,h_210/v1/imagens/product/B43951/b35d5826-d10c-4775-b1e7-8e7c3c64642f-escova-desembaraco-sophie.png
3: Escova para Desembaraçar Match, 54, 2, https://res.cloudinary.com/beleza-na-web/image/upload/w_1500,f_auto,fl_progressive,q_auto:eco,w_210,h_210/v1/imagens/product/B43939/84414a26-d1c8-4aa6-8908-91eb6ff06b84-match-escova-para-desembaracar-43939.png
4: Escova Polvo O Boticário Match, 54, 3, https://res.cloudinary.com/beleza-na-web/image/upload/w_1500,f_auto,fl_progressive,q_auto:eco,w_210,h_210/v1/imagens/product/B53787/dbd6125f-414e-4e2e-b82f-a34685d5ad0c-bot-53787-escova-cabelos-polvo-frontal-01.jpg
5: Leave-in Glossy Cuide-se Bem Melissa 150ml, 32, 4, https